In [4]:
from collections import namedtuple
import numpy as np
import cv2
# define the `Detection` object
Detection = namedtuple("Detection", ["image_path", "gt", "pred"])


def bb_intersection_over_union(boxA, boxB):
	# determine the (x, y)-coordinates of the intersection rectangle
	xA = max(boxA[0], boxB[0])
	yA = max(boxA[1], boxB[1])
	xB = min(boxA[2], boxB[2])
	yB = min(boxA[3], boxB[3])
	# compute the area of intersection rectangle
	interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
	# compute the area of both the prediction and ground-truth
	# rectangles
	boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
	boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
	# compute the intersection over union by taking the intersection
	# area and dividing it by the sum of prediction + ground-truth
	# areas - the intersection area
	iou = interArea / float(boxAArea + boxBArea - interArea)
	# return the intersection over union value
	return iou

examples = {
    Detection("Data/sample3.jpg", [[50, 50, 100, 100]], [[48, 48, 102, 102]]),
}


Precision: 0.50


In [5]:
def evaluate_recall(predicted_boxes, predicted_classes, ground_truth_boxes, ground_truth_classes, iou_threshold=0.5):
    """Evaluate recall for a single image."""
    true_positives = 0
    matched_ground_truths = set()

    for i, gt_box in enumerate(ground_truth_boxes):
        gt_class = ground_truth_classes[i]
        matched = False

        for j, pred_box in enumerate(predicted_boxes):
            if calculate_iou(pred_box, gt_box) >= iou_threshold and predicted_classes[j] == gt_class:
                true_positives += 1
                matched_ground_truths.add(i)
                matched = True
                break

    false_negatives = len(ground_truth_boxes) - len(matched_ground_truths)

    # Calculate recall
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0.0
    return recall

# Example usage
predicted_boxes = [[50, 50, 100, 100], [30, 30, 70, 70]]  # x1, y1, x2, y2
predicted_classes = [1, 2]
ground_truth_boxes = [[48, 48, 102, 102], [60, 60, 110, 110]]
ground_truth_classes = [1, 2]

recall = evaluate_recall(predicted_boxes, predicted_classes, ground_truth_boxes, ground_truth_classes)
print(f"Recall: {recall:.2f}")


Recall: 0.50


In [6]:
import numpy as np

def calculate_iou(box1, box2):
    """Compute Intersection over Union (IoU) between two bounding boxes."""
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])

    inter_area = max(0, x2 - x1) * max(0, y2 - y1)
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union_area = box1_area + box2_area - inter_area

    return inter_area / union_area if union_area > 0 else 0.0

def compute_ap(pred_boxes, pred_scores, pred_classes, gt_boxes, gt_classes, iou_threshold=0.5):
    """Compute Average Precision (AP) for a single class."""
    # Sort predictions by confidence score
    sorted_indices = np.argsort(-np.array(pred_scores))
    pred_boxes = [pred_boxes[i] for i in sorted_indices]
    pred_classes = [pred_classes[i] for i in sorted_indices]

    tp = np.zeros(len(pred_boxes))
    fp = np.zeros(len(pred_boxes))
    matched_gt = set()

    for i, pred_box in enumerate(pred_boxes):
        pred_class = pred_classes[i]
        best_iou = 0
        best_gt_idx = -1

        for j, gt_box in enumerate(gt_boxes):
            if j in matched_gt:
                continue
            if pred_class == gt_classes[j]:
                iou = calculate_iou(pred_box, gt_box)
                if iou > best_iou:
                    best_iou = iou
                    best_gt_idx = j

        if best_iou >= iou_threshold:
            tp[i] = 1
            matched_gt.add(best_gt_idx)
        else:
            fp[i] = 1

    # Calculate precision and recall
    cum_tp = np.cumsum(tp)
    cum_fp = np.cumsum(fp)
    precision = cum_tp / (cum_tp + cum_fp)
    recall = cum_tp / len(gt_boxes)

    # Interpolate precision-recall curve and calculate AP
    precision = np.concatenate(([0], precision, [0]))
    recall = np.concatenate(([0], recall, [1]))
    for i in range(len(precision) - 1, 0, -1):
        precision[i - 1] = max(precision[i - 1], precision[i])
    indices = np.where(recall[1:] != recall[:-1])[0]
    ap = np.sum((recall[indices + 1] - recall[indices]) * precision[indices + 1])
    return ap

def compute_map50(predictions, ground_truths, num_classes, iou_threshold=0.5):
    """Compute mAP@50 for all classes."""
    ap_values = []
    for c in range(num_classes):
        # Filter predictions and ground truths by class
        pred_boxes = [pred["box"] for pred in predictions if pred["class"] == c]
        pred_scores = [pred["score"] for pred in predictions if pred["class"] == c]
        pred_classes = [pred["class"] for pred in predictions if pred["class"] == c]

        gt_boxes = [gt["box"] for gt in ground_truths if gt["class"] == c]
        gt_classes = [gt["class"] for gt in ground_truths if gt["class"] == c]

        if len(gt_boxes) > 0:
            ap = compute_ap(pred_boxes, pred_scores, pred_classes, gt_boxes, gt_classes, iou_threshold)
            ap_values.append(ap)

    return np.mean(ap_values) if ap_values else 0.0

# Example usage
predictions = [
    {"box": [50, 50, 100, 100], "score": 0.9, "class": 0},
    {"box": [30, 30, 70, 70], "score": 0.8, "class": 1},
]
ground_truths = [
    {"box": [48, 48, 102, 102], "class": 0},
    {"box": [28, 28, 72, 72], "class": 1},
]
num_classes = 2

map50 = compute_map50(predictions, ground_truths, num_classes)
print(f"mAP@50: {map50:.2f}")


mAP@50: 1.00
